In [1]:
import torch

from torch.utils.data import Dataset
from torchvision.transforms import Compose

import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_breast_cancer

In [2]:
class Cancer_Dataset(Dataset):
    
    def __init__(self,transforms=None):
        X,y = load_breast_cancer(return_X_y=True)
        
        self.mu = torch.tensor(np.mean(X,0)).view(1,-1)
        self.sd = torch.tensor(np.std(X,0)).view(1,-1)
        
        self.x = torch.DoubleTensor(X)
        self.y = torch.LongTensor(y)
        
        self.transforms = transforms
        self.len = X.shape[0]
        
    def __getitem__(self,index):
        sample = self.x[index,:].view(1,-1),self.y[index].view(1,-1)
        
        if self.transforms:
            sample = self.transforms(sample)
        
        return sample
    
    def __len__(self):
        return self.len

In [3]:
class Standard_Normalizer(object):
    def __init__(self,mu,std):
        self.mu = mu
        self.std = std
        
    def __call__(self,sample):
        x = sample[0]
        y = sample[1]
        
        x = (x-self.mu)/self.std
        sample = x,y
        return sample

In [4]:
class Logistic_Regression(object):
    
    def __init__(self,num_input):
        self.number_of_features = num_input
        self.bias = torch.tensor([[1]],dtype=torch.float64,requires_grad=True)
        self.weights = torch.randn((num_input,1),dtype=torch.float64,requires_grad=True)
        
    def __call__(self,x):
        y = 1/(1+torch.exp(-torch.mm(x,self.weights+self.bias)))
        return torch.cat((1-y,y),1)
    
    def update(self,alpha=.1):
        with torch.no_grad():
            self.weights -= self.weights.grad * alpha
            self.weights.grad.zero_()

            self.bias -= self.bias.grad * alpha
            self.bias.grad.zero_()

In [5]:
criterion = torch.nn.NLLLoss()

In [6]:
dataset = Cancer_Dataset()
normalizer = Standard_Normalizer(dataset.mu,dataset.sd)

In [7]:
model = Logistic_Regression(30)

In [37]:
i = 0
print_every = 2
for epoch in range(10):
    l = []
    for x,y in dataset:
        x,y = normalizer((x,y))
        y_pred = model(x)
        loss = criterion(y_pred,y.squeeze(1))

        loss.backward()

        l.append(loss.detach().item())
        
        model.update()

    if (epoch+1) % print_every == 0:
        print("Epoch {}: loss={}".format(epoch+1,np.mean(l)))

Epoch 2: loss=-0.9872234501425587
Epoch 4: loss=-0.9872482354956467
Epoch 6: loss=-0.987273558881205
Epoch 8: loss=-0.9872991657707604
Epoch 10: loss=-0.987324840611743


In [38]:
with torch.no_grad():
    predictions = []
    for x,y in dataset:
        x,y = normalizer((x,y))
        y_pred = model(x)
        predictions.append(y_pred.argmax().item())

In [39]:
print("Accuracy",np.mean(dataset.y.numpy() == np.array(predictions)))

Accuracy 0.9894551845342706
